# Compute Radiomics

In this example notebook we use [PyRadiomics](https://github.com/AIM-Harvard/pyradiomics) to
compute various type of radiomics features. We use some
[LCTSC](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=24284539) data from the
Cancer Imaging Archive which has already been converted using PyDicer for demonstration purposes.

In [1]:
try:
    from pydicer import PyDicer
except ImportError:
    !pip install pydicer
    from pydicer import PyDicer

from pathlib import Path

from pydicer.utils import fetch_converted_test_data

from pydicer.utils import load_object_metadata

## Fetch data

LCTSC data prepared for this example are downloaded and stored into a `testdata_lctsc` directory.
We will use this for our PyDicer working directory.

In [2]:
working_directory = fetch_converted_test_data("./testdata_lctsc", dataset="LCTSC")

## Initialise PyDicer object

Using the working directory containing the LCTSC test data.

In [3]:
pydicer = PyDicer(working_directory)

## Compute Default Radiomics

By default, PyDicer will compute only first-order radiomics features. Radiomics are computed for
each structure available in the dataset using the image data of images linked to those structures.

In [4]:
pydicer.analyse.compute_radiomics()

  0%|          | 0/10 [00:00<?, ?objects/s, Compute Radiomics]

 10%|█         | 1/10 [00:00<00:04,  1.81objects/s, Compute Radiomics]

 20%|██        | 2/10 [00:01<00:04,  1.75objects/s, Compute Radiomics]

 30%|███       | 3/10 [00:01<00:04,  1.70objects/s, Compute Radiomics]

 40%|████      | 4/10 [00:02<00:03,  1.68objects/s, Compute Radiomics]

...

 70%|███████   | 7/10 [00:03<00:01,  1.80objects/s, Compute Radiomics]

 80%|████████  | 8/10 [00:04<00:01,  1.74objects/s, Compute Radiomics]

 90%|█████████ | 9/10 [00:05<00:00,  1.72objects/s, Compute Radiomics]

100%|██████████| 10/10 [00:05<00:00,  1.79objects/s, Compute Radiomics]

100%|██████████| 10/10 [00:05<00:00,  1.76objects/s, Compute Radiomics]

## Fetch computed Radiomics

Use the `get_all_computed_radiomics_for_dataset` function to fetch all radiomics features computed
in the last step.

The `.head()` function on a Pandas DataFrame output the first 5 rows for inspection.

In [5]:
# Display the DataFrame of radiomics computed
df_radiomics = pydicer.analyse.get_all_computed_radiomics_for_dataset()
df_radiomics.head()

,Contour,Patient,ImageHashedUID,StructHashedUID,firstorder|10Percentile,firstorder|90Percentile,firstorder|Energy,firstorder|Entropy,firstorder|InterquartileRange,firstorder|Kurtosis,...,firstorder|Mean,firstorder|Median,firstorder|Minimum,firstorder|Range,firstorder|RobustMeanAbsoluteDeviation,firstorder|RootMeanSquared,firstorder|Skewness,firstorder|TotalEnergy,firstorder|Uniformity,firstorder|Variance
3,Esophagus,LCTSC-Test-S1-101,666be6,cc682f,-9.0,74.0,6.379739e+06,2.563380,35.25,55.979763,...,33.382764,42.0,-745.0,940.0,15.951183,70.379070,-5.961993,1.642731e+08,0.232723,3838.804579
1,Heart,LCTSC-Test-S1-101,666be6,cc682f,-14.0,68.0,8.486142e+07,2.343017,31.00,16.880718,...,34.078823,44.0,-577.0,806.0,14.250737,53.881633,-2.753651,2.185112e+09,0.273513,1741.864194
2,Lung_L,LCTSC-Test-S1-101,666be6,cc682f,-871.0,-502.0,3.907161e+10,4.313815,148.00,6.423229,...,-729.756106,-778.0,-1000.0,1357.0,67.906500,747.249117,1.747786,1.006062e+12,0.069585,25837.268599
4,Lung_R,LCTSC-Test-S1-101,666be6,cc682f,-889.0,-480.0,4.790689e+10,4.414603,167.00,5.712595,...,-735.842185,-789.0,-1000.0,1215.0,76.876692,755.465238,1.617379,1.233563e+12,0.063675,29264.004413
0,SpinalCord,LCTSC-Test-S1-101,666be6,cc682f,-19.0,70.0,6.830546e+06,2.638360,43.00,12.650592,...,27.051730,27.0,-144.0,573.0,18.213519,49.364661,1.348291,1.758810e+08,0.205445,1705.073671


## Specify Radiomics to Compute

PyDicer used the popular `pyradiomics` library to compute radiomics. So, you may specify any
radiomics features provided in that library to be computed. See the [pyradiomics documentation for
a list of radiomics features
available](https://pyradiomics.readthedocs.io/en/latest/features.html).

In this example, we specify all `shape` features as well as `first-order` features to be computed.

In [6]:
# Import pyradiomics feature modules
from radiomics import (
    firstorder,
    shape,
)

# Prepare a dict of features to compute grouped by class
first_order_features = firstorder.RadiomicsFirstOrder.getFeatureNames()
shape_features = shape.RadiomicsShape.getFeatureNames()
compute_radiomics = {
    "firstorder": [f for f in first_order_features if not first_order_features[f]],
    "shape": [f for f in shape_features if not shape_features[f]],
}

# Pass the dict to the compute the radiomics
pydicer.analyse.compute_radiomics(radiomics=compute_radiomics)

# Fetch the computed radiomics and output the first few rows
df_radiomics = pydicer.analyse.get_all_computed_radiomics_for_dataset()
df_radiomics.head()

  0%|          | 0/10 [00:00<?, ?objects/s, Compute Radiomics]

 10%|█         | 1/10 [00:01<00:13,  1.46s/objects, Compute Radiomics]

 20%|██        | 2/10 [00:03<00:13,  1.71s/objects, Compute Radiomics]

 30%|███       | 3/10 [00:05<00:12,  1.79s/objects, Compute Radiomics]

 40%|████      | 4/10 [00:07<00:11,  2.00s/objects, Compute Radiomics]

...

 80%|████████  | 8/10 [00:14<00:03,  2.00s/objects, Compute Radiomics]

 90%|█████████ | 9/10 [00:16<00:01,  1.93s/objects, Compute Radiomics]

100%|██████████| 10/10 [00:18<00:00,  1.82s/objects, Compute Radiomics]

100%|██████████| 10/10 [00:18<00:00,  1.83s/objects, Compute Radiomics]

,Contour,Patient,ImageHashedUID,StructHashedUID,firstorder|10Percentile,firstorder|90Percentile,firstorder|Energy,firstorder|Entropy,firstorder|InterquartileRange,firstorder|Kurtosis,...,shape|Maximum2DDiameterColumn,shape|Maximum2DDiameterRow,shape|Maximum2DDiameterSlice,shape|Maximum3DDiameter,shape|MeshVolume,shape|MinorAxisLength,shape|Sphericity,shape|SurfaceArea,shape|SurfaceVolumeRatio,shape|VoxelVolume
3,Esophagus,LCTSC-Test-S1-101,666be6,cc682f,-9.0,74.0,6.379739e+06,2.563380,35.25,55.979763,...,223.924778,222.309084,23.619884,224.231209,3.193220e+04,50.733471,0.408351,11919.810387,0.373285,3.316495e+04
1,Heart,LCTSC-Test-S1-101,666be6,cc682f,-14.0,68.0,8.486142e+07,2.343017,31.00,16.880718,...,163.733306,127.040344,151.241053,164.152138,7.513182e+05,100.977462,0.781808,51121.049949,0.068042,7.526486e+05
2,Lung_L,LCTSC-Test-S1-101,666be6,cc682f,-871.0,-502.0,3.907161e+10,4.313815,148.00,6.423229,...,236.649430,242.714238,190.429595,245.982024,1.801684e+06,171.702531,0.568443,125964.905979,0.069915,1.801747e+06
4,Lung_R,LCTSC-Test-S1-101,666be6,cc682f,-889.0,-480.0,4.790689e+10,4.414603,167.00,5.712595,...,231.159117,247.961931,186.213760,248.126635,2.159880e+06,172.513299,0.582948,138614.093158,0.064177,2.161386e+06
0,SpinalCord,LCTSC-Test-S1-101,666be6,cc682f,-19.0,70.0,6.830546e+06,2.638360,43.00,12.650592,...,300.128718,313.974459,26.529436,314.315983,7.050233e+04,32.966235,0.418454,19722.946105,0.279749,7.217496e+04


You can also set a specific subset of features like this:

In [7]:
# Specify features to compute grouped by class
compute_radiomics = {
    "firstorder": ["Maximum", "Minimum", "Mean", "Median"],
    "shape": ["SurfaceArea", "VoxelVolume"],
}

# Pass the dict to the compute the radiomics
pydicer.analyse.compute_radiomics(radiomics=compute_radiomics)

df_radiomics = pydicer.analyse.get_all_computed_radiomics_for_dataset()
df_radiomics.head()

  0%|          | 0/10 [00:00<?, ?objects/s, Compute Radiomics]

 10%|█         | 1/10 [00:01<00:12,  1.44s/objects, Compute Radiomics]

 20%|██        | 2/10 [00:03<00:13,  1.65s/objects, Compute Radiomics]

 30%|███       | 3/10 [00:05<00:12,  1.74s/objects, Compute Radiomics]

 40%|████      | 4/10 [00:07<00:11,  1.96s/objects, Compute Radiomics]

...

 80%|████████  | 8/10 [00:14<00:03,  1.96s/objects, Compute Radiomics]

 90%|█████████ | 9/10 [00:16<00:01,  1.90s/objects, Compute Radiomics]

100%|██████████| 10/10 [00:17<00:00,  1.77s/objects, Compute Radiomics]

100%|██████████| 10/10 [00:17<00:00,  1.79s/objects, Compute Radiomics]

,Contour,Patient,ImageHashedUID,StructHashedUID,firstorder|Maximum,firstorder|Minimum,firstorder|Mean,firstorder|Median,shape|SurfaceArea,shape|VoxelVolume
3,Esophagus,LCTSC-Test-S1-101,666be6,cc682f,195.0,-745.0,33.382764,42.0,11919.810387,3.316495e+04
1,Heart,LCTSC-Test-S1-101,666be6,cc682f,229.0,-577.0,34.078823,44.0,51121.049949,7.526486e+05
2,Lung_L,LCTSC-Test-S1-101,666be6,cc682f,357.0,-1000.0,-729.756106,-778.0,125964.905979,1.801747e+06
4,Lung_R,LCTSC-Test-S1-101,666be6,cc682f,215.0,-1000.0,-735.842185,-789.0,138614.093158,2.161386e+06
0,SpinalCord,LCTSC-Test-S1-101,666be6,cc682f,429.0,-144.0,27.051730,27.0,19722.946105,7.217496e+04


## Track Metadata

When analysing your radiomic features, it may be useful to have certain metadata available from
either the image or structure set. You can specify which DICOM header tags to extract metadata for
and these will be stored alongside the radiomic feature values.

In the cell below, we recompute our radiomics and store the `PatientSex` header value from the
image series and the `StudyDate` value from the structure set.

In [8]:
# Compute the radiomics specifying the meta data to keep
pydicer.analyse.compute_radiomics(
    radiomics=compute_radiomics,
    image_meta_data=["PatientSex"],
    structure_meta_data=["StudyDate"]
)

# Fetch the results and display the first rows
df_radiomics = pydicer.analyse.get_all_computed_radiomics_for_dataset()
df_radiomics.head()

  0%|          | 0/10 [00:00<?, ?objects/s, Compute Radiomics]

 10%|█         | 1/10 [00:01<00:12,  1.44s/objects, Compute Radiomics]

 20%|██        | 2/10 [00:03<00:13,  1.65s/objects, Compute Radiomics]

 30%|███       | 3/10 [00:05<00:12,  1.77s/objects, Compute Radiomics]

 40%|████      | 4/10 [00:07<00:11,  1.96s/objects, Compute Radiomics]

...

 80%|████████  | 8/10 [00:14<00:03,  1.96s/objects, Compute Radiomics]

 90%|█████████ | 9/10 [00:16<00:01,  1.88s/objects, Compute Radiomics]

100%|██████████| 10/10 [00:17<00:00,  1.75s/objects, Compute Radiomics]

100%|██████████| 10/10 [00:17<00:00,  1.78s/objects, Compute Radiomics]

,Contour,Patient,ImageHashedUID,StructHashedUID,firstorder|Maximum,firstorder|Minimum,firstorder|Mean,firstorder|Median,shape|SurfaceArea,shape|VoxelVolume,struct|StudyDate,img|PatientSex
3,Esophagus,LCTSC-Test-S1-101,666be6,cc682f,195.0,-745.0,33.382764,42.0,11919.810387,3.316495e+04,20040303,M
1,Heart,LCTSC-Test-S1-101,666be6,cc682f,229.0,-577.0,34.078823,44.0,51121.049949,7.526486e+05,20040303,M
2,Lung_L,LCTSC-Test-S1-101,666be6,cc682f,357.0,-1000.0,-729.756106,-778.0,125964.905979,1.801747e+06,20040303,M
4,Lung_R,LCTSC-Test-S1-101,666be6,cc682f,215.0,-1000.0,-735.842185,-789.0,138614.093158,2.161386e+06,20040303,M
0,SpinalCord,LCTSC-Test-S1-101,666be6,cc682f,429.0,-144.0,27.051730,27.0,19722.946105,7.217496e+04,20040303,M
